In [29]:
#import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, date_format, round, when, expr
#from google.colab import drive
from datetime import datetime
#drive.mount('/content/drive')

spark=SparkSession.builder.appName('Propwise').config("spark.jars", r"C:\Users\Asad\Documents\Propwise Challenge\postgresql-42.7.6.jar").getOrCreate()

#file_path = '/content/drive/MyDrive/Propwise Challenge/transactions-2025-06-07.csv'
file_path = r"C:\Users\Asad\Documents\Propwise Challenge\transactions-2025-06-07.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

#df.printSchema()

"""Cleaning and transforming data"""

def transform(df):

  ### Remove null and duplicate records
  cleaned_df = df.na.drop()
  cleaned_df = cleaned_df.dropDuplicates()
  #df = df.dropna()
  #df = df.drop_duplicates()

  ### Standardize column names
  for oldCol in cleaned_df.columns:
    cleaned_df = cleaned_df.withColumnRenamed(oldCol, oldCol.lower().replace(' ','_'))
  #df.columns = [x.lower().replace(' ','_') for x in df.columns]

  ### Parse dates

  # Convert instance_date column to type timestamp
  cleaned_df = cleaned_df.withColumn('instance_date_timestamp', to_timestamp(col('instance_date'), 'yyyy-MM-dd HH:mm:ss'))

  # Place the date and time in their own new columns
  cleaned_df = cleaned_df.withColumn('date_col', date_format(col('instance_date_timestamp'), 'yyyy-MM-dd'))
  cleaned_df = cleaned_df.withColumn('time_col', date_format(col('instance_date_timestamp'), 'HH:mm:ss'))

  # Drop the instance_date column
  #cleaned_df = cleaned_df.drop('instance_date')

  #df = df.withColumn('date', to_timestamp(col('date'), 'MM/dd/yyyy'))
  #if df.columns.str.contains('date', case= False).any():
   # df.columns = pd.to_datetime(df.columns)

  ### Parse numeric columns
  for column in ["trans_value", "procedure_area", "actual_area"]:
    cleaned_df = cleaned_df.withColumn(column, round(col(column).cast('double'),2))

  ### New columns (price_per_sqm, price_per_room, budget_tier)
  cleaned_df = cleaned_df.withColumn('price_per_sqm', round(col('trans_value')/col('procedure_area'),2))
  #cleaned_df = cleaned_df.withColumn('property_age', 2025 - col('date_col'))
  cleaned_df = cleaned_df.withColumn('price_per_room', round(col("trans_value") / when(col("rooms_en") == "Studio", 1).otherwise(expr("try_cast(substr(rooms_en, 1, 1) as int)")), 2))
  cleaned_df = cleaned_df.withColumn('budget_tier', when(col("trans_value")<=600000, "Low Budget").when((col("trans_value")>600000) & (col("trans_value")<=2500000), "Medium Budget").otherwise("High Budget"))



  return cleaned_df




In [30]:
cleaned_df = transform(df)
cleaned_df.show()

+------------------+-------------------+--------+--------------------+-------------+---------------+-----------+--------------------+------------+---------------+-----------+--------------+-----------+--------+-------+--------------------+--------------------+--------------------+-----------+------------+--------------------+--------------------+-----------------------+----------+--------+-------------+--------------+-------------+
|transaction_number|      instance_date|group_en|        procedure_en|is_offplan_en|is_free_hold_en|   usage_en|             area_en|prop_type_en|prop_sb_type_en|trans_value|procedure_area|actual_area|rooms_en|parking|    nearest_metro_en|     nearest_mall_en| nearest_landmark_en|total_buyer|total_seller|   master_project_en|          project_en|instance_date_timestamp|  date_col|time_col|price_per_sqm|price_per_room|  budget_tier|
+------------------+-------------------+--------+--------------------+-------------+---------------+-----------+----------------

In [31]:
def load(cleaned_df, table_name):
  # PostgreSQL info
  postgresql_url = r"jdbc:postgresql://localhost:5432/UAE_Real_Estate"
  properties = {
      "user" : "postgres",
      "password" : "123",
      "driver" : "org.postgresql.Driver"
  }

  # Write DataFrame to PostgreSQL table
  cleaned_df.write.jdbc(url=postgresql_url, table=table_name, mode="overwrite", properties=properties)
  

cleaned_df = transform(df)
load(cleaned_df, "dubai_real_estate_transactions")



spark.stop()

In [32]:
output_csv_path = r"C:\Users\Asad\Documents\Propwise Challenge\cleaned_transactions_output"
cleaned_df.write.option("header", True).mode("overwrite").csv(output_csv_path)



Py4JJavaError: An error occurred while calling o2358.csv.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Co